In [1]:
#match_away_team_info table
from glob import glob
import pandas as pd
# Find all directories under tennis_data/
date_folders = glob('tennis_data/*')

column_away= ['match_id', 'player_id','full_name']
files_away= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_away.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/away_team*.parquet') if '_score' not in f]
    )
df_away_team_info = [pd.read_parquet(file, columns=column_away) for file in files_away]
df_away_team_info = pd.concat(df_away_team_info, ignore_index=True)

In [2]:
#matchhometeaminfo table
column_home= ['match_id', 'player_id', 'full_name']

# Create an empty list to store all files
files_home= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_home.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/home_team*.parquet') if '_score' not in f]
    )
df_home_team_info = [pd.read_parquet(file, columns=column_home) for file in files_home]
df_home_team_info = pd.concat(df_home_team_info, ignore_index=True)

In [3]:
# Match_event_info table
files_event = []
column_event= ['match_id','winner_code','start_datetime']
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_event.extend(
        glob(f'{folder}/data/raw/raw_match_parquet/event*.parquet')
    )
event_info = [pd.read_parquet(file, columns=column_event) for file in files_event]
event_info = pd.concat(event_info, ignore_index=True)

In [4]:
tour_info = []

# Loop through each date folder and load .parquet files
for folder in date_folders:
    tour_info.extend(
        glob(f'{folder}/data/raw/raw_match_parquet/tournament*.parquet')
    )

df_tour_info = [pd.read_parquet(file) for file in tour_info]
df_tour_info = pd.concat(df_tour_info, ignore_index=True)

In [75]:
import pandas as pd

# Step 1: Filter matches with valid winner
df_event_win = event_info[event_info['winner_code'].isin([1, 2])]

# Step 2: Join with player info
df_home_win = df_event_win[df_event_win['winner_code'] == 1].merge(
    df_home_team_info[['match_id', 'full_name']], on='match_id', how='left'
)
df_home_win['winner_name'] = df_home_win['full_name']

df_away_win = df_event_win[df_event_win['winner_code'] == 2].merge(
    df_away_team_info[['match_id', 'full_name']], on='match_id', how='left'
)
df_away_win['winner_name'] = df_away_win['full_name']

# Combine all winning matches
df_winners = pd.concat([df_home_win, df_away_win], ignore_index=True)

# Step 3: Join with tournament info
df_winners = df_winners.merge(
    df_tour_info[['match_id', 'tournament_name']], on='match_id', how='left'
)

# Step 4: Add match month
df_winners['month'] = pd.to_datetime(df_winners['start_datetime'], unit='s').dt.to_period('M')

# Step 5: Count unique tournaments per player per month
df_winners_unique = df_winners[['winner_name', 'month', 'tournament_name']].drop_duplicates()
df_counts = df_winners_unique.groupby(['winner_name', 'month']).size().reset_index(name='tournament_wins')

# Step 6: Find max
top_winner = df_counts.loc[df_counts['tournament_wins'].idxmax()]

# Find maximum number of wins
max_wins = df_counts['tournament_wins'].max()

# Filter all players with that max
top_players = df_counts[df_counts['tournament_wins'] == max_wins]

print(top_players.sort_values(['month', 'winner_name']))


         winner_name    month  tournament_wins
2439      Naw, Hazem  2024-02                7
576   Cekirge, Kuzey  2024-03                7
2887    Ratti, Lucio  2024-03                7
